In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm
import pandas as pd
import torch.optim.lr_scheduler

In [27]:
# 데이터 전처리 (CIFAR10은 3채널 이미지)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# CIFAR10 학습 및 테스트 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# CIFAR10의 targets는 리스트로 되어 있으므로 tensor로 변환
targets = torch.tensor(train_dataset.targets)

num_samples_per_class = 500
selected_indices = []

# 0부터 9까지 각 클래스별로 인덱스를 추출하고, 랜덤하게 num_samples_per_class개 선택
for class_label in range(10):
    # 해당 클래스의 인덱스 추출
    indices = (targets == class_label).nonzero(as_tuple=True)[0]
    # 인덱스를 랜덤하게 섞은 후 원하는 개수만 선택
    selected = indices[torch.randperm(len(indices))[:num_samples_per_class]]
    selected_indices.extend(selected.tolist())

# 선택한 인덱스만 남긴 서브셋 생성
train_dataset = Subset(train_dataset, selected_indices)

In [28]:
class MyLayer(nn.Module):
    def __init__(self, in_channels, out_channels, sequence):
        super(MyLayer, self).__init__()
        if sequence == 1:
            self.layer = nn.Sequential(
                nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
        elif sequence == 2:
            self.layer = nn.Sequential(
                nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
                nn.ReLU(),
                nn.BatchNorm2d(out_channels),
            )
        elif sequence == 3:
            self.layer = nn.Sequential(
                nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
                nn.LeakyReLU(),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        return self.layer(x)


class MyNet(nn.Module):
    def __init__(self, sequence):
        super(MyNet, self).__init__()
        self.sequence = sequence

        self.layer1 = MyLayer(3, 16, sequence)
        self.layer2 = MyLayer(16, 32, sequence)
        self.layer3 = MyLayer(32, 64, sequence)
        self.layer4 = MyLayer(64, 128, sequence)
        self.layer5 = MyLayer(128, 64, sequence)
        self.layer6 = MyLayer(64, 32, sequence)
        self.layer7 = MyLayer(32, 16, sequence)
        self.layer8 = MyLayer(16, 10, sequence)

        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.classifier = nn.Linear(in_features=10 * 2 * 2, out_features=10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.pool(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.pool(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [39]:
def experiment(seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Accuracies = []
    num_epochs = 50

    def worker_init_fn(worker_id):
        # 메인 프로세스의 torch.random.seed()로부터 worker에 고유 seed 부여
        worker_seed = torch.initial_seed() % 2**32  # torch.initial_seed()는 메인 seed와 worker_id에 기반하여 결정됨
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    # 매 실험이 동일한 조건에서 시작하기 위해 시드 고정
    print(f"[Seed : {seed}] Experiment Start")

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8, pin_memory=True, worker_init_fn = worker_init_fn)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8, pin_memory=True, worker_init_fn = worker_init_fn)
    models = (model1:=MyNet(1), model2:=MyNet(2), model3:=MyNet(3))
    scaler = torch.amp.grad_scaler.GradScaler()

    for model in models:
        # 시드 고정
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.0195, momentum=0.91, weight_decay=0.0)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
            #Train Code
        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"{'Conv->BN->ReLU' if model.sequence == 1 else 'Conv->ReLU->BN' if model.sequence == 2 else 'Conv->LeakyReLU->BN'} with epoch {epoch + 1}")
            for (data, target) in progress_bar:
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()

                with torch.amp.autocast(device_type="cuda", dtype=torch.bfloat16):
                    output = model(data)
                    loss = criterion(output, target)
                epoch_loss += loss.item()
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                progress_bar.set_postfix({"Loss": loss.item()})
            scheduler.step()
            print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader)}, lr = {optimizer.param_groups[0]['lr']}")

    for model in models:
        #Test Code
        model.to(device)
        model.eval()
        correct = 0
        with torch.no_grad():
            for _, (data, target) in enumerate(test_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                correct += (output.argmax(dim=1) == target).sum().item()
        top1_acc = 100 * correct / len(test_loader.dataset)
        if model.sequence == 1:
            Accuracies.append(top1_acc)
        elif model.sequence == 2:
            Accuracies.append(top1_acc)
        else:
            Accuracies.append(top1_acc)
        print(f"sequence : {'Conv->BN->ReLU' if model.sequence == 1 else 'Conv->ReLU->BN' if model.sequence == 2 else 'Conv->LeakyReLU->BN'}, Top1 Accuracy: {top1_acc:.2f}%")
        del model
        torch.cuda.empty_cache()

    return Accuracies


if __name__ == "__main__":
    results = []
    num_experiments = 3
    for i in range(num_experiments):
        seed = random.randint(0, 100000)
        results.append(experiment(seed))

[Seed : 99584] Experiment Start


Conv->BN->ReLU with epoch 1: 100%|██████████| 20/20 [00:04<00:00,  4.24it/s, Loss=1.69]


Epoch 1, Loss: 1.9801063299179078, lr = 0.0195


Conv->BN->ReLU with epoch 2: 100%|██████████| 20/20 [00:04<00:00,  4.24it/s, Loss=1.38]


Epoch 2, Loss: 1.580157995223999, lr = 0.0195


...

Conv->LeakyReLU->BN with epoch 49: 100%|██████████| 20/20 [00:04<00:00,  4.23it/s, Loss=0.00688]


Epoch 49, Loss: 0.005032408609986305, lr = 0.00019500000000000002


Conv->LeakyReLU->BN with epoch 50: 100%|██████████| 20/20 [00:04<00:00,  4.21it/s, Loss=0.00576]

Epoch 50, Loss: 0.004590147454291582, lr = 0.00019500000000000002


sequence : Conv->BN->ReLU, Top1 Accuracy: 63.38%
sequence : Conv->ReLU->BN, Top1 Accuracy: 59.39%
sequence : Conv->LeakyReLU->BN, Top1 Accuracy: 59.42%


In [40]:
# 결과 정리
dfs = []
for i in range(num_experiments):
    print(f"[Experiment Result]")
    Accuracies = results[i]
    sorted_Accuracies = sorted(Accuracies, reverse=True)
    ranks = [sorted_Accuracies.index(Accuracies[i]) + 1 for i in range(len(Accuracies))]

    index = ["Accuracy", "Rank"]
    data = {
        "Conv->BN->ReLU": (Accuracies[0], int(ranks[0])),
        "Conv->ReLU->BN": (Accuracies[1], ranks[1]),
        "Conv->LeakyReLU->BN": (Accuracies[2], ranks[2])
    }

    df = pd.DataFrame(data, index=index)
    dfs.append(df)

    print(df)
    print("\n")

print("[Averages]")
average_df = dfs[0]
for i in range(1, num_experiments):
    average_df += dfs[i]
average_df /= num_experiments
print(average_df)

[Experiment Result]
          Conv->BN->ReLU  Conv->ReLU->BN  Conv->LeakyReLU->BN
Accuracy           62.32           59.56                60.97
Rank                1.00            3.00                 2.00


[Experiment Result]
          Conv->BN->ReLU  Conv->ReLU->BN  Conv->LeakyReLU->BN
Accuracy           62.23           56.54                59.87
Rank                1.00            3.00                 2.00


[Experiment Result]
          Conv->BN->ReLU  Conv->ReLU->BN  Conv->LeakyReLU->BN
Accuracy           63.38           59.39                59.42
Rank                1.00            3.00                 2.00


[Averages]
          Conv->BN->ReLU  Conv->ReLU->BN  Conv->LeakyReLU->BN
Accuracy       62.643333       58.496667            60.086667
Rank            1.000000        3.000000             2.000000
